In [ ]:
# Coded by G. Petricca (@gmrpetricca)

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
from matplotlib import cm as color_map
from datetime import date, datetime
import matplotlib.image as image
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib as mpl
from PIL import Image
import pandas as pd
import numpy as np
import math

# some basic station data for the subtitle
RX_STATION = "Stornoway, Scotland"
EQUIPMENT = "9 Element Yagi - Airspy R2 - SDR Sharp - SpectrumLab"

# define maximum value for the bar chart, to have a uniform frame of reference for all months
yscalemax = 100

# address of the general .xlsx data file
xlsx_address = "D:\MeteorsXXXX.xlsx" # modify this to match the name and the location of the source file (if they are in the same folder, then only the name will have to be changed)
file_year = int(xlsx_address[10:][:-5])

# ask the user for a month to plot
while True:
    try:
        userchoice = int(input("Which month do you want to visualize? (Insert a number bewteen 1 and 12)")) 
        
        months = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}
        
        if 1 <= userchoice <= 12:
            month_name = months[userchoice]
            print("\n" + "You have selected " + str(month_name) + " " + str(file_year) + ".\n")
            break
        else:
            print("Input is out of range, please insert a valid one.")
    except:
        print("That's not a number! Please insert a valid entry.")

# read raw data from excel file and create a dataframe
dfs = pd.read_excel(xlsx_address, sheet_name=month_name)

# erase the first named column
dfs = dfs.drop(dfs.columns[0], axis=1) 

# reset index to start with 1 instead that with 0
dfs.index = np.arange(1,len(dfs)+1)

# calculate the hourly average and create a second dataframe
avgrow = dfs.mean(axis=1) 
avgrow = round(avgrow,1)

dfs_avg = pd.DataFrame(avgrow, columns=['Avg'])

# calculate the daily total and create a third dataframe
sumcol = dfs.sum(axis=0)

dfs_sum = pd.DataFrame(sumcol, columns=['Sum'])

# calculate dataframes maximums
dfsmax = np.nanmax(dfs.values)
dfs_avg_max = dfs_avg.to_numpy().max()
dfs_sum_max = dfs_sum.to_numpy().max()

# then transpose the totals one
dfs_sum = dfs_sum.transpose()

# get code run day in the month
today = date.today()
year = int(today.strftime("%y"))
month = int(today.strftime("%m"))
day = int(today.strftime("%d"))

# get first all-zeros column index and differentiate between 
# filled months and still incomplete months
try:
    zeros = [i for i, e in enumerate(sumcol) if e == 0][0] + 1
except IndexError as e:
    # print(e) # in case of errors
    zeros = []

# check if the zero list is empty or not, and assembles the data for the final chart
if not zeros:
    dfs = dfs.replace([0],0.1) # this is to avoid coloring past zeros in the same tone of future zeros
else:
    dfs_values = dfs.iloc[:,:zeros-1]
    dfs_zeros = dfs.iloc[:,zeros-1:]

    if datetime(year, month, day) < datetime(file_year, userchoice, zeros):
        dfs_values = dfs_values.replace([0],0.1) # this is to avoid coloring past zeros in the same tone of future zeros
    else:
        pass
    
    # create final dataframe
    dfs = pd.concat([dfs_values, dfs_zeros], axis = 1)

# start of the first plot - grid monthly view
dfs = dfs.replace(np.nan,0)

# calculate the total monthly sum 
tot = dfs.astype(int).values.sum()
    
# setup figure
fig, ax = plt.subplots(figsize=(32, 20))

# set cmap
cmap = plt.get_cmap('viridis')
cmap.set_under('lightslategrey')

# add colorbar based on maximum hourly rate
if dfsmax >= 60:
    # round bar max to nearest ten
    brvmax_norm = int(math.ceil(dfsmax / 10.0)) * 10
    
    # normalize colormap/colorbar using a power law to make smaller values more visible in case of high hourly rates
    normal = mpl.colors.PowerNorm(gamma=0.85, vmin=0, vmax=brvmax_norm)
    
    sm = mpl.cm.ScalarMappable(norm=normal, cmap=cmap)
    sm.set_array([])

    cb_pl = fig.colorbar(sm, ticks=ticker.MaxNLocator(nbins=10), format=mpl.ticker.ScalarFormatter(), aspect=50, pad=-0.08)
    cb_pl.set_label('Hourly Meteor Count', color="darkblue", fontsize=20, labelpad=20)
    cb_pl.ax.tick_params(labelsize=13)
    
    chart = ax.imshow(dfs, cmap=cmap, interpolation='nearest', vmin=0.0000000001, vmax=brvmax_norm, norm=normal)

else:
    # round bar max to nearest ten
    brvmax = int(math.ceil(dfsmax / 10.0)) * 10
    
    # in this case keep a linear progression for the colormap/colorbar
    chart = ax.imshow(dfs, cmap=cmap, interpolation='nearest', vmin=0.0000000001, vmax=brvmax)
    
    cb = fig.colorbar(chart, ax=ax, aspect=50, pad=-0.08)
    cb.set_label('Hourly Meteor Count', color="darkblue", fontsize=20, labelpad=20)
    cb.ax.tick_params(labelsize=13)
    tick_locator = ticker.MaxNLocator(nbins=10)
    cb.locator = tick_locator
    cb.update_ticks()

divider = make_axes_locatable(ax)

# setup day count and logo/text positions based on the month
if month_name in ['January', 'March', 'May', 'July', 'August', 'October', 'December']:
    ndays = 31
    logox = 210
    textx = 0.110
    cropx = 165
elif month_name in ['April', 'June', 'September', 'November']:
    ndays = 30
    logox = 230
    textx = 0.120
    cropx = 190
else:
    ndays = 29
    logox = 280
    textx = 0.142
    cropx = 240

# set x-y axis details
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xticks(np.arange(0, ndays, step=1))
ax.set_xticklabels(np.arange(1, ndays+1, step=1))
ax.set_yticks(np.arange(0, 24, step=1))
ax.tick_params(axis=u'both', which=u'both', length=0, pad=10)

# set x-y axis labels
ax.set_xlabel('Day of the Month', color="darkblue", fontsize=20, labelpad=15)
ax.set_ylabel('Hour of the Day', color="darkblue", fontsize=20, labelpad=10)

# loop over data in the dataframe and create text annotations.
dfs_val = dfs.astype(int)
data = dfs_val.values
for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        if data[y, x] == dfsmax:
            color ='crimson'
            size = 17
        else:
            color ='white'
            size = 15
        txt = ax.text(x, y, data[y, x], ha='center', va='center', color=color, size=size)
        
# average left plot
axleft = divider.append_axes("left", size=1.0, pad=1.0, sharey=ax)

brv_avg_max = int(math.ceil(dfs_avg_max / 10.0)) * 10

axleft.imshow(dfs_avg, cmap=cmap, interpolation='nearest', vmin=0.0000000001, vmax=brv_avg_max)

# adjust plot labels
axleft.tick_params(axis=u'y', which=u'both', length=0, pad=7)
axleft.axes.xaxis.set_ticks([])
axleft.set_ylabel('Average Hourly Meteor Rate', color="darkblue", fontsize=20, labelpad=15)

# loop over data in the dataframe and create text annotations
data = np.ma.masked_equal(dfs_avg.values, 0)
for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        if data[y, x] == dfs_avg_max:
            color ='crimson'
            size = 17
        else:
            color ='white'
            size = 15
        axleft.text(x, y, data[y, x], ha='center', va='center', color=color, size=size)
        
# total bottom plot
axbottom = divider.append_axes("bottom", size=1.0, pad=0.85, sharex=ax)

brv_sum_max = int(math.ceil(dfs_sum_max / (dfs_sum_max+100))) * (dfs_sum_max+100)

axbottom.imshow(dfs_sum, cmap=cmap, interpolation='nearest', vmin=0.0000000001, vmax=brv_sum_max)

# adjust plot labels
axbottom.tick_params(axis=u'x', which=u'both', length=0, pad=7)
axbottom.axes.yaxis.set_ticks([])
axbottom.set_xlabel('Daily Total Meteors', color="darkblue", fontsize=20, labelpad=15)

# loop over data in the dataframe and create text annotations
dfs_val = dfs_sum.astype(int)
data = dfs_val.values
for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        if data[y, x] == dfs_sum_max:
            color ='crimson'
            size = 17
        else:
            color ='white'
            size = 15
        axbottom.text(x, y, data[y, x], ha='center', va='center', color=color, size=size)

# figure additional text
fig.text(0.50, 0.980, "Registered Meteor Echoes Hourly Totals for " + str(months[userchoice]) + " " + str(file_year), ha="center", va="center", size=25, color="darkblue")
fig.text(0.50, 0.960, "Echoes received from: " + RX_STATION + " - Equipment: " + EQUIPMENT, ha="center", va="center", fontsize=15, color="black")
fig.text(0.858, 0.012, "Infographic by Giuseppe Petricca         @gmrpetricca", ha="right", va="center", size=12, color="black")
fig.text(textx, 0.961, "Total Meteors: " + str(tot), ha="left", va="center", size=20, color="firebrick")

# figure additional logos
im1 = image.imread('img/HWlogo.png') # if you have a personal logo, you can add it here
fig.figimage(im1, 1847, 1365, zorder=3)

im2 = image.imread('img/matplotliblogo.png')
fig.figimage(im2, logox, 35, zorder=3)

im3 = image.imread('img/twitterlogo.png')
fig.figimage(im3, 1867, 8, zorder=3)

im4 = image.imread('img/airspylogo.png')
fig.figimage(im4, logox+5, 100, zorder=3)

im5 = image.imread('img/pandaslogo.png')
fig.figimage(im5, 1680, 1373, zorder=3)

# save and plot the first figure
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("GridChart_" + str(month_name) + "_" + str(file_year) + "_Meteor_Count.png")
plt.close()

# crop to create final first image
img = Image.open("GridChart_" + str(month_name) + "_" + str(file_year) + "_Meteor_Count.png")
cropped_img = img.crop((cropx,0,2070,1440))
cropped_img.save("GridChart_" + str(month_name) + "_" + str(file_year) + "_Meteor_Count.png")

# close the first plot
plt.close()

# start of the second plot - hourly totals bar chart

# get lenght of month from number of columns in the dataframe
chart_lenght = len(dfs.columns)

# put the dataframe in a list
element_list=[]
for i in range(1,chart_lenght+1):
    list_elements = dfs[i].tolist()
    element_list.append(list_elements)

# setup second figure
fig, ax = plt.subplots(figsize=(32, 20))

# generate the x y coordinates for the bar chart
x_coords = np.arange(1, (chart_lenght*24)+1, 1)
y_coords = [y for sublist in element_list for y in sublist]
y_max = int(math.ceil(dfsmax / 10.0)) * 10

# plot the bar chart
bar = ax.bar(x_coords, y_coords, align='center', width=1.0)

# some cosmetic refinement
positions = np.arange(0, len(x_coords), 24)
labels = np.arange(1, chart_lenght+1, 1)
plt.xticks(positions, labels)

ax.set_facecolor('dodgerblue')
ax.patch.set_alpha(0.05)
ax.margins(x=0.0007)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_ylim([0,yscalemax])
ax.set_yticks(np.arange(0, yscalemax+0.0000000001, 5))
ax.grid(color='dimgray', ls = '-.', lw = 0.33)

# gradientbar colormap courtesy of: https://stackoverflow.com/questions/38830250/how-to-fill-matplotlib-bars-with-a-gradient
def gradientbars(bars):
    grad = np.atleast_2d(np.linspace(0,1,256)).T
    #grad = grad[::-1] # to reverse the colormap array
    ax = bars[0].axes
    lim = ax.get_xlim()+ax.get_ylim()
    for bar in bars:
        bar.set_zorder(1)
        bar.set_facecolor("none")
        x,y = bar.get_xy()
        w, h = bar.get_width(), bar.get_height()
        ax.imshow(grad, extent=[x,x+w,y,y+h], aspect="auto", zorder=0)
    ax.axis(lim)

gradientbars(bar)

# set x y axes labels
ax.set_xlabel("Day of the Month", color="darkblue", fontsize=20, labelpad=15)
ax.set_ylabel("Meteor Echoes Hourly Totals", color="darkblue", fontsize=20, labelpad=10)

# figure additional text
fig.text(0.50, 0.980, "Registered Meteor Echoes Hourly Totals for " + str(months[userchoice]) + " " + str(file_year), ha="center", va="center", size=27, color="darkblue")
fig.text(0.50, 0.960, "Echoes received from: " + RX_STATION + " - Equipment: " + EQUIPMENT, ha="center", va="center", fontsize=17, color="black")
fig.text(0.996, 0.012, "Infographic by Giuseppe Petricca         @gmrpetricca", ha="right", va="center", size=13, color="black")
fig.text(0.0305, 0.950, "Total Meteors: " + str(tot), ha="left", va="center", size=20, color="firebrick")

# figure additional logos
im1 = image.imread('img/HWlogo.png') # if you have a personal logo, you can add it here
fig.figimage(im1, 2152, 1365, zorder=3)

im2 = image.imread('img/matplotliblogo.png')
fig.figimage(im2, 2149, 1205, zorder=3)

im3 = image.imread('img/twitterlogo.png')
fig.figimage(im3, 2177, 8, zorder=3)

im4 = image.imread('img/airspylogo.png')
fig.figimage(im4, 2150, 1285, zorder=3)

im5 = image.imread('img/pandaslogo.png')
fig.figimage(im5, 2149, 1125, zorder=3)

# save and plot the second figure
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("BarChart_" + str(month_name) + "_" + str(file_year) + "_Meteor_Count.png")

# close the second plot
plt.close()

# conclude the program
print(str(month_name) + " " + str(file_year) + " charts have been created.")